In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.cluster import DBSCAN
from flask import Flask, jsonify
import plotly.express as px
import dash
from dash import dcc, html
from datetime import datetime

df = pd.read_csv('IOT-temp.csv')
df.head()

,id,room_id/id,noted_date,temp,out/in
0,__export__.temp_log_196134_bd201015,Room Admin,08-12-2018 09:30,29,In
1,__export__.temp_log_196131_7bca51bc,Room Admin,08-12-2018 09:30,29,In
2,__export__.temp_log_196127_522915e3,Room Admin,08-12-2018 09:29,41,Out
3,__export__.temp_log_196128_be0919cf,Room Admin,08-12-2018 09:29,41,Out
4,__export__.temp_log_196126_d30b72fb,Room Admin,08-12-2018 09:29,31,In


In [2]:
def preprocess_data(df):
    # Konversi noted_date ke datetime
    df['noted_date'] = pd.to_datetime(df['noted_date'], format='%d-%m-%Y %H:%M')

    # Feature engineering: Ekstrak jam dari noted_date
    df['hour'] = df['noted_date'].dt.hour

    # Normalisasi suhu
    scaler = StandardScaler()
    df['temp_scaled'] = scaler.fit_transform(df[['temp']])

    return df

In [3]:
df_processed = preprocess_data(df)

In [4]:
def detect_anomalies(df):
    # Isolation Forest
    iso_forest = IsolationForest(contamination=0.1, random_state=42)
    df['anomaly_iso'] = iso_forest.fit_predict(df[['temp_scaled', 'hour']])
    df['anomaly_iso'] = df['anomaly_iso'].map({1: 'Normal', -1: 'Anomaly'})

    # DBSCAN
    dbscan = DBSCAN(eps=0.5, min_samples=2)
    df['cluster_dbscan'] = dbscan.fit_predict(df[['temp_scaled', 'hour']])
    df['anomaly_dbscan'] = df['cluster_dbscan'].apply(lambda x: 'Anomaly' if x == -1 else 'Normal')

    return df

In [5]:
df_results = detect_anomalies(df_processed)

In [6]:
app_flask = Flask(__name__)

@app_flask.route('/anomalies', methods=['GET'])
def get_anomalies():
    anomalies = df_results[df_results['anomaly_iso'] == 'Anomaly'][['id', 'temp', 'noted_date']].to_dict('records')
    return jsonify(anomalies)

In [7]:
app_dash = dash.Dash(__name__)

app_dash.layout = html.Div([
    html.H1("Temperature Anomaly Dashboard"),
    dcc.Graph(
        id='scatter-plot',
        figure=px.scatter(
            df_results,
            x='noted_date',
            y='temp',
            color='anomaly_iso',
            title='Temperature Anomalies (Isolation Forest)',
            labels={'anomaly_iso': 'Anomaly Status'}
        )
    ),
    dcc.Graph(
        id='cluster-plot',
        figure=px.scatter(
            df_results,
            x='noted_date',
            y='temp',
            color='cluster_dbscan',
            title='Temperature Clustering (DBSCAN)',
            labels={'cluster_dbscan': 'Cluster'}
        )
    )
])

In [8]:
df_results.to_csv('anomaly_results.csv', index=False)
print("Anomaly detection completed. Results saved to 'anomaly_results.csv'.")
print(df_results[['id', 'temp', 'noted_date', 'anomaly_iso', 'anomaly_dbscan']])

Anomaly detection completed. Results saved to 'anomaly_results.csv'.
                                        id  temp          noted_date  \
0      __export__.temp_log_196134_bd201015    29 2018-12-08 09:30:00   
1      __export__.temp_log_196131_7bca51bc    29 2018-12-08 09:30:00   
2      __export__.temp_log_196127_522915e3    41 2018-12-08 09:29:00   
3      __export__.temp_log_196128_be0919cf    41 2018-12-08 09:29:00   
4      __export__.temp_log_196126_d30b72fb    31 2018-12-08 09:29:00   
...                                    ...   ...                 ...   
97601   __export__.temp_log_91076_7fbd08ca    31 2018-07-28 07:07:00   
97602  __export__.temp_log_147733_62c03f31    31 2018-07-28 07:07:00   
97603  __export__.temp_log_100386_84093a68    31 2018-07-28 07:06:00   
97604  __export__.temp_log_123297_4d8e690b    31 2018-07-28 07:06:00   
97605  __export__.temp_log_133741_32958703    31 2018-07-28 07:06:00   

      anomaly_iso anomaly_dbscan  
0          Normal         Norma

In [9]:
if __name__ == '__main__':
    import threading
    threading.Thread(target=lambda: app_flask.run(debug=False, port=5000)).start()
    app_dash.run(debug=False, port=8050)

 * Serving Flask app '__main__'
 * Debug mode: off
